In [1]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'rail-demo-367913'

In [2]:
REGION = 'europe-west4'

In [3]:
from google.cloud import storage

import pandas as pd
import numpy as np

In [4]:
gcs = storage.Client(project = PROJECT_ID)

In [5]:
BUCKET = PROJECT_ID

---
## Create Storage Bucket
Check to see if bucket already exist and create if missing:
- [GCS Python Client](https://cloud.google.com/python/docs/reference/storage/latest/google.cloud.storage.client.Client)

In [6]:
if not gcs.lookup_bucket(BUCKET):
    bucketDef = gcs.bucket(BUCKET)
    bucket = gcs.create_bucket(bucketDef, project=PROJECT_ID, location=REGION)
    print(f'Created Bucket: {gcs.lookup_bucket(BUCKET).name}')
else:
    bucketDef = gcs.bucket(BUCKET)
    print(f'Bucket already exist: {bucketDef.name}')

Bucket already exist: rail-demo-367913


In [7]:
print(f'Review the storage bucket in the console here:\nhttps://console.cloud.google.com/storage/browser/{PROJECT_ID};tab=objects&project={PROJECT_ID}')

Review the storage bucket in the console here:
https://console.cloud.google.com/storage/browser/rail-demo-367913;tab=objects&project=rail-demo-367913


---
<a id = 'permissions'></a>
## Service Account & Permissions

This notebook instance is running as a service account in GCP.  This service account will also be used to run other services in Vertex AI like training jobs and pipelines.  The service account will need permission to interact with object in Cloud Storage which requires the role ([roles/storage.objectAdmin](https://cloud.google.com/storage/docs/access-control/iam-roles)).  

In [6]:
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'595378874286-compute@developer.gserviceaccount.com'

In [7]:
!gcloud services enable cloudresourcemanager.googleapis.com

In [8]:
!gcloud projects get-iam-policy $PROJECT_ID --filter="bindings.members:$SERVICE_ACCOUNT" --format='table(bindings.role)' --flatten="bindings[].members"

ROLE
roles/owner
roles/storage.objectAdmin


In [9]:
!pip install kfp -U -q
!pip install google-cloud-pipeline-components -U -q

---
## Update AIPlatform Package:

The `google-cloud-aiplatform` package updates frequently.  Update it for latest functionality.

- [aiplatform Python Client](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform)
- [GitHub Repo for api-common-protos](https://github.com/googleapis/api-common-protos)

For a better understanding of the Vertex AI APIs client, version, and layers please review the tip here [aiplatform_notes.md](../Tips/aiplatform_notes.md).

In [12]:
!pip install googleapis-common-protos -U -q
!pip install google-cloud-aiplatform -U -q

# MODEL IMPLEMENTATION

In [10]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

gcs = storage.Client(project = PROJECT_ID)
BUCKET = PROJECT_ID

In [11]:
# source data
RAIL_PROJECT = PROJECT_ID
RAIL_DATASET = 'rail-demo-367913'

In [12]:
import tensorflow as tf
import numpy as np

In [13]:
list(bucketDef.list_blobs())

NameError: name 'bucketDef' is not defined

In [ ]:
"""#to download files from cloud storage
bucket_name = "rail-demo-367913"

storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)

# When you have your files in a subfolder of the bucket.
my_prefix = "saved_arrays_multiclass/" # the name of the subfolder
blobs = bucket.list_blobs(prefix = my_prefix, delimiter = '/')

for blob in blobs:
    if(blob.name != my_prefix): # ignoring the subfolder itself 
        file_name = blob.name.replace(my_prefix, "")
        blob.download_to_filename(file_name) # download the file to the machine
        df = np.load(file_name) # load the data
        print(df)
"""

In [14]:
import numpy as np
import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Input, Conv2DTranspose, Concatenate
import random
from keras.callbacks import ModelCheckpoint
from keras import backend as K
import cv2
import os
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [15]:
DIM = 512
EPOCHS = 1
PATIENCE =1
BATCH_SIZE = 8
OPTIMIZER = 'adam'
N_CHANNELS = 3
N_CLASSES = 3
DIM = 512
MODEL_NAME = "unet_multiclass.h5"

In [16]:

src_path = "saved_arrays_multiclass"
train_images = np.load(src_path + "/train_images.npy") #FIXME aggiungere tutte le immagini
train_masks = np.load(src_path + "/train_masks.npy")
val_images = np.load(src_path + "/val_images.npy")
val_masks = np.load(src_path + "/val_masks.npy")
test_images = np.load(src_path + "/test_images.npy")
test_masks = np.load(src_path + "/test_masks.npy")
print("train images", train_images.shape)
print("train masks", train_masks.shape)
print("val images", val_images.shape)
print("test images", test_images.shape)


train images (858, 512, 512, 3)
train masks (858, 512, 512, 3)
val images (167, 512, 512, 3)
test images (118, 512, 512, 3)


In [17]:

train_images = train_images/255.0
val_images = val_images/255.0
test_images = test_images/255.0


In [18]:

checkpoint_path = "checkpoints" #where to save the model checkpoints
if not os.path.exists(checkpoint_path):
    os.mkdir(checkpoint_path)



### UNET

In [19]:
# Building Unet by dividing encoder and decoder into blocks

from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from tensorflow.keras.optimizers import Adam
from keras.layers import Activation, MaxPool2D, Concatenate


def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)   #Not in the original network. 
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)  #Not in the original network
    x = Activation("relu")(x)

    return x

#Encoder block: Conv block followed by maxpooling


def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p   

#Decoder block
#skip features gets input from encoder for concatenation

def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

#Build Unet using the blocks
def build_unet(input_shape, n_classes):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024) #Bridge

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    

    outputs = Conv2D(n_classes, 1, padding="same", activation="softmax")(d4)  #Change the activation based on n_classes
    

    model = Model(inputs, outputs, name="U-Net")
    return model

In [20]:

model = build_unet((DIM,DIM,3), n_classes=3)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

2022-12-21 10:31:08.922508: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-21 10:31:08.931471: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-21 10:31:08.931733: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-21 10:31:08.932899: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "U-Net"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 512, 512, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 512, 512, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 512, 512, 64) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [21]:

def build_callbacks():
    checkpointer = [
          EarlyStopping(monitor="val_loss", patience=PATIENCE, restore_best_weights=True),
          ModelCheckpoint(checkpoint_path + '/'+ MODEL_NAME,  monitor="val_loss", save_best_only=True) #best on iou
    ]
    return checkpointer

In [22]:
model.fit(train_images, train_masks,  batch_size=BATCH_SIZE, verbose=1, epochs=EPOCHS, validation_data=(val_images, val_masks), callbacks = build_callbacks())


2022-12-21 10:31:14.841626: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-12-21 10:31:16.935419: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005
2022-12-21 10:31:20.170155: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 529.50MiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-12-21 10:31:20.170285: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 556.38MiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-12-21 10:31:20.244938: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory tryi

108/108 [==============================] - 123s 1s/step - loss: 2.0462 - accuracy: 0.7033 - val_loss: 3023.9531 - val_accuracy: 0.2067


In [23]:
def plot_img_and_masks(raw, mask):
    pred = model.predict(np.expand_dims(raw, 0))
    pred1 = np.argmax(pred, axis=-1)[0,:,:]
    pred_msk = np.stack((pred1,) * 3, axis=-1)
    pred_msk[(pred_msk[:, :, 0] == 1) & (pred_msk[:, :, 1] == 1) & (pred_msk[:, :, 2] == 1)] = (255,0,0)
    pred_msk[(pred_msk[:, :, 0] == 2) & (pred_msk[:, :, 1] == 2) & (pred_msk[:, :, 2] == 2)] = (0,255,0)

    mask1 = np.argmax(mask, axis=-1)
    target_msk = np.stack((mask1,) * 3, axis=-1)
    target_msk[(target_msk[:, :, 0] == 1) & (target_msk[:, :, 1] == 1) & (target_msk[:, :, 2] == 1)] = (255,0,0)
    target_msk[(target_msk[:, :, 0] == 2) & (target_msk[:, :, 1] == 2) & (target_msk[:, :, 2] == 2)] = (0,255,0)

    raw = np.float32(raw)
    pred_msk = np.float32(pred_msk)
    target_msk = np.float32(target_msk)
    raw = cv2.cvtColor(raw, cv2.COLOR_RGB2BGR)
    pred_msk = cv2.cvtColor(pred_msk, cv2.COLOR_RGB2BGR)
    target_msk = cv2.cvtColor(target_msk, cv2.COLOR_RGB2BGR)
    combined = np.concatenate([raw, pred_msk, target_msk], axis=1)
    return combined

In [24]:

sample_dir = "samples_multiclass" #where to save the predictions
if not os.path.exists(sample_dir):
    os.mkdir(sample_dir)

for i in range(test_images.shape[0]):
    test_img = test_images[i]
    test_mask = test_masks[i]
    combined = plot_img_and_masks(test_img, test_mask)
    cv2.imwrite(sample_dir + "/" + str(i) + '.jpg', combined * 255)

In [25]:
test_images.shape

(118, 512, 512, 3)

In [29]:

"""Quantitative measures"""

from evaluation_multiclass import mean_iou_test, dice_coeff, pixel_accuracy

print("TEST RESULTS")
pred_masks = model.predict(test_images) #(n examples,h,w,3)

test_masks1 = np.argmax(test_masks, axis=-1) #(n examples,h,w)
pred_masks1 = np.argmax(pred_masks, axis=-1)

iou, iou_classes = mean_iou_test(test_masks1, pred_masks1,num_classes=3)
print("mean iou", iou)
print(iou_classes)
dice, dice_classes = dice_coeff(test_masks1, pred_masks1,num_classes=3)
print("dice coeff.", dice)
print(dice_classes)
acc, acc_classes = pixel_accuracy(test_masks1, pred_masks1,num_classes=3)
print("pixel acc.", acc)
print(acc_classes)


TEST RESULTS
mean iou 0.028325346323320552
[0.008921519008785992, 0.069644213255069, 0.00641030670610666]
dice coeff. 0.053547870128536755
[0.01768525864638298, 0.130219398921689, 0.01273895281753828]
pixel acc. 0.3342898308479598
[0.008929896181743417, 0.24782087490094887, 0.7461187214611872]
